In [4]:
import pandas as pd
import datetime as dt


path_const= r'C:\Users\fnafisa\WORKSPACE\professional\Project PoC\Input\data\Construction materials\segregated\{}'
path_price_index= r'C:\Users\fnafisa\WORKSPACE\professional\Project PoC\Input\data\Construction materials\warehouses.xlsx'

path_oil_brent= r'C:\Users\fnafisa\WORKSPACE\professional\Project PoC\Input\data\Oil\Brent Oil Futures Historical Data; monthly.csv'
path_TASI= r'C:\Users\fnafisa\WORKSPACE\professional\Project PoC\Input\data\Markets\Tadawul All Share Historical Data monthly.csv'
path_Gold= r'C:\Users\fnafisa\WORKSPACE\professional\Project PoC\Input\data\gold\Gold Futures Historical Data monthly.csv'
path_interest_rate= r'C:\Users\fnafisa\WORKSPACE\professional\Project PoC\Input\data\Fed fund rates\FEDFUNDS.csv'


# Create date column for both dataframes:
# this function takes a string object which is in a date format and covert it to a date object
def str_to_datetime(s):
  split = s.split('/')
  month, day, year = int(split[0]), int(split[1]), int(split[2])
  return dt.date(year=year, month=month, day=day)

def str_to_datetime_ii(s):
  split = s.split('-')
  year, month, day = int(split[0]), int(split[1]), int(split[2])
  return dt.date(year=year, month=month, day=day)

In [5]:
#           oil
df_oil_brent= pd.read_csv(path_oil_brent)
df_oil_brent['Date'] = df_oil_brent['Date'].apply(str_to_datetime)
df_oil_brent.index = df_oil_brent.pop('Date')
# we keep only the needed columns
df_oil_brent= pd.DataFrame(df_oil_brent['Price'])
df_oil_brent.columns= ['oil_brent']

#           TASI
df_TASI= pd.read_csv(path_TASI)
df_TASI['Date'] = df_TASI['Date'].apply(str_to_datetime)
df_TASI.index = df_TASI.pop('Date')
# we keep only the needed columns
df_TASI= pd.DataFrame(df_TASI['Price'])
df_TASI.columns= ['tasi']

#           gold
df_gold= pd.read_csv(path_Gold)
df_gold['Date'] = df_gold['Date'].apply(str_to_datetime)
df_gold.index = df_gold.pop('Date')
# we keep only the needed columns
df_gold= pd.DataFrame(df_gold['Price'])
df_gold.columns= ['gold']

#           interest rates
df_interest_rates= pd.read_csv(path_interest_rate)
df_interest_rates['Date'] = df_interest_rates['DATE'].apply(str_to_datetime_ii)
df_interest_rates.index = df_interest_rates.pop('Date')
# we keep only the needed columns
df_interest_rates= pd.DataFrame(df_interest_rates['FEDFUNDS'])
df_interest_rates.columns= ['interest_rate']

In [6]:
df_const_mat= pd.read_excel(path_price_index)
df_const_mat.columns= ['cat.', 'type', 'price', 'month name', 'year', 'month', 'unit']
df_const_mat= df_const_mat.loc[:,['type', 'month','year', 'price']]
const_types= list(set(df_const_mat['type']))

dict_price_index= {}

for t in const_types:
    dict_price_index[t]= df_const_mat[df_const_mat['type'] == t]
    
for df_temp in dict_price_index:
    # prepare the TARGET
    dict_price_index[df_temp].pop('type')
    dict_price_index[df_temp]['date'] = dict_price_index[df_temp]['month'].map(str)+ '/'  + '1' + '/' + dict_price_index[df_temp]['year'].map(str)
    dict_price_index[df_temp]['date'] = dict_price_index[df_temp]['date'].apply(str_to_datetime)
    dict_price_index[df_temp].index = dict_price_index[df_temp].pop('date')
    dict_price_index[df_temp]= pd.DataFrame(dict_price_index[df_temp]['price'])

    # join data frames to prepare the PREDICTORS
    df_res = pd.merge(dict_price_index[df_temp], df_oil_brent, how='left', left_index=True, right_index=True)
    df_res = pd.merge(df_res,df_TASI, how='left', left_index=True, right_index=True)
    df_res = pd.merge(df_res,df_gold, how='left', left_index=True, right_index=True)
    df_res = pd.merge(df_res,df_interest_rates, how='left', left_index=True, right_index=True)
    
    # add historical data
    df_res['historical']= df_res['price'].shift(1)
    df_res['historical'].fillna(value=df_res['historical'].mean(), inplace=True)
    df_res['element_name']= df_temp

    filename= df_temp+'.xlsx'
    filepath= path_const.format(filename)
    print(filepath)
    df_res.to_excel(filepath)#, index=False)

pd.DataFrame(const_types).to_excel(path_const.format('type names.xlsx'), index=False)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_3268\1592248770.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dict_price_index[df_temp]['date'] = dict_price_index[df_temp]['month'].map(str)+ '/'  + '1' + '/' + dict_price_index[df_temp]['year'].map(str)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_3268\1592248770.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dict_price_index[df_temp]['date'] = dict_price_index[df_temp]['date'].apply(str_to_datetime)
C:\Users\Administrator\AppData\Loc

C:\Users\fnafisa\WORKSPACE\professional\Project PoC\Input\data\Construction materials\segregated\ألمنيوم (حلق جنب سعودي).xlsx
C:\Users\fnafisa\WORKSPACE\professional\Project PoC\Input\data\Construction materials\segregated\خرسانة مقاوم 250ك.xlsx
C:\Users\fnafisa\WORKSPACE\professional\Project PoC\Input\data\Construction materials\segregated\حديد تسليح 18مم وطني.xlsx


C:\Users\Administrator\AppData\Local\Temp\ipykernel_3268\1592248770.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dict_price_index[df_temp]['date'] = dict_price_index[df_temp]['month'].map(str)+ '/'  + '1' + '/' + dict_price_index[df_temp]['year'].map(str)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_3268\1592248770.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dict_price_index[df_temp]['date'] = dict_price_index[df_temp]['date'].apply(str_to_datetime)
C:\Users\Administrator\AppData\Loc

C:\Users\fnafisa\WORKSPACE\professional\Project PoC\Input\data\Construction materials\segregated\كيابل كهربائية 50مم وطني.xlsx
C:\Users\fnafisa\WORKSPACE\professional\Project PoC\Input\data\Construction materials\segregated\كيابل ربط الحديد صيني.xlsx
C:\Users\fnafisa\WORKSPACE\professional\Project PoC\Input\data\Construction materials\segregated\حديد تسليح 8مم وطني.xlsx


C:\Users\Administrator\AppData\Local\Temp\ipykernel_3268\1592248770.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dict_price_index[df_temp]['date'] = dict_price_index[df_temp]['month'].map(str)+ '/'  + '1' + '/' + dict_price_index[df_temp]['year'].map(str)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_3268\1592248770.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dict_price_index[df_temp]['date'] = dict_price_index[df_temp]['date'].apply(str_to_datetime)
C:\Users\Administrator\AppData\Loc

...

C:\Users\Administrator\AppData\Local\Temp\ipykernel_3268\1592248770.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dict_price_index[df_temp]['date'] = dict_price_index[df_temp]['month'].map(str)+ '/'  + '1' + '/' + dict_price_index[df_temp]['year'].map(str)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_3268\1592248770.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dict_price_index[df_temp]['date'] = dict_price_index[df_temp]['date'].apply(str_to_datetime)
C:\Users\Administrator\AppData\Loc

C:\Users\fnafisa\WORKSPACE\professional\Project PoC\Input\data\Construction materials\segregated\أسلاك كهربائية 4مم وطني.xlsx
C:\Users\fnafisa\WORKSPACE\professional\Project PoC\Input\data\Construction materials\segregated\كيابل كهربائية 70مم وطني.xlsx
C:\Users\fnafisa\WORKSPACE\professional\Project PoC\Input\data\Construction materials\segregated\كيابل كهربائية 35مم وطني.xlsx
C:\Users\fnafisa\WORKSPACE\professional\Project PoC\Input\data\Construction materials\segregated\مخلوط ( رمل و بحص).xlsx


C:\Users\Administrator\AppData\Local\Temp\ipykernel_3268\1592248770.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dict_price_index[df_temp]['date'] = dict_price_index[df_temp]['month'].map(str)+ '/'  + '1' + '/' + dict_price_index[df_temp]['year'].map(str)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_3268\1592248770.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dict_price_index[df_temp]['date'] = dict_price_index[df_temp]['date'].apply(str_to_datetime)
C:\Users\Administrator\AppData\Loc

C:\Users\fnafisa\WORKSPACE\professional\Project PoC\Input\data\Construction materials\segregated\كيابل كهربائية 10مم وطني.xlsx
C:\Users\fnafisa\WORKSPACE\professional\Project PoC\Input\data\Construction materials\segregated\خرسانة عادي 250ك.xlsx
C:\Users\fnafisa\WORKSPACE\professional\Project PoC\Input\data\Construction materials\segregated\خرسانة عادي 350ك.xlsx
C:\Users\fnafisa\WORKSPACE\professional\Project PoC\Input\data\Construction materials\segregated\حديد تسليح 6مم وطني.xlsx
C:\Users\fnafisa\WORKSPACE\professional\Project PoC\Input\data\Construction materials\segregated\أسلاك كهربائية 6مم وطني.xlsx


C:\Users\Administrator\AppData\Local\Temp\ipykernel_3268\1592248770.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dict_price_index[df_temp]['date'] = dict_price_index[df_temp]['month'].map(str)+ '/'  + '1' + '/' + dict_price_index[df_temp]['year'].map(str)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_3268\1592248770.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dict_price_index[df_temp]['date'] = dict_price_index[df_temp]['date'].apply(str_to_datetime)
